In [124]:

import pandas as pd
%load_ext autoreload
%autoreload 2

from pathlib import Path

import seaborn as sns
import sys,os
sys.path.append(os.path.realpath('data_managment/'))
from sampler import PairwiseSampler
from s3_connector import download_dataset_version


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:

data_dir = Path('../raw_data/')
save_dir = Path('data/')
path = Path('data/train_dataset.fth')
# dataset = Dataset(data_dir, save_dir)
# dataset.prepare_and_save_data()


In [56]:
df = pd.read_feather(path)
df.head()


,index,id,cell_id,cell_type,source,rank,pct_rank,ancestor_id,parent_id
0,0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many he...,0,0.000000,945aea18,None
1,1,00001756c60be8,448eb224,markdown,**Импортируем необходимые для работы функции и...,1,0.017241,945aea18,None
2,2,00001756c60be8,2a9e43d6,code,import numpy as np\nimport pandas as pd\nimpor...,2,0.034483,945aea18,None
3,3,00001756c60be8,7e2f170a,markdown,**Подключаем предупреждения**,3,0.051724,945aea18,None
4,4,00001756c60be8,038b763d,code,import warnings\nwarnings.filterwarnings('igno...,4,0.068966,945aea18,None


In [86]:
import numpy as np
from tqdm import tqdm


def generate_triplet(df, mode='train'):
  triplets = []
  ids = df.id.unique()
  random_drop = np.random.random(size=10000)>0.9
  count = 0

  for id, df_tmp in tqdm(df.groupby('id')):
    df_tmp_markdown = df_tmp[df_tmp['cell_type']=='markdown']

    df_tmp_code = df_tmp[df_tmp['cell_type']=='code']
    df_tmp_code_rank = df_tmp_code['rank'].values
    df_tmp_code_cell_id = df_tmp_code['cell_id'].values

    # для каждой маркдаун ячейки в ноутбуке
    # создаются лэйблы - это массив, в котором единица стоит на номере, соответствующему
    # элементу со следующим ранком кодовой ячейки
    # Таким образом для каждого маркдауна выдается следующая кодовая ячейка

    # далее для каждой кодовой ячейки создается триплет
    # Если это следующая ячейка от маркдауна (label = 1), то создается
    # массив [id маркдауна, id кода, 1]
    # Или же если если это тест, то создаются триплеты со всеми кодовыми ячейками
    # Если же это не следующая кодовая ячейка и не тест, то с вероятностью 10%
    # создается триплет с этой кодовой ячейкой

    for cell_id, rank in df_tmp_markdown[['cell_id', 'rank']].values:
      labels = np.array([(r==(rank+1)) for r in df_tmp_code_rank]).astype('int')
      for cid, label in zip(df_tmp_code_cell_id, labels):
        count += 1
        md_source = df_tmp_markdown[df_tmp_markdown['cell_id'] == cell_id].iloc[0].source
        cd_source = df_tmp_code[df_tmp_code['cell_id'] == cid].iloc[0].source
        if label==1:        
          triplets.append( [md_source, cd_source, label] )

#           triplets.append( [cell_id, cid, label] )
          # triplets.append( [cid, cell_id, label] )
        elif mode == 'test':
          triplets.append( [md_source, cd_source, label] )


#           triplets.append( [cell_id, cid, label] )
          # triplets.append( [cid, cell_id, label] )
        elif random_drop[count%10000]:
          triplets.append( [md_source, cd_source, label] )


#           triplets.append( [cell_id, cid, label] )
          # triplets.append( [cid, cell_id, label] )

  return triplets


In [129]:
ntb_id = '00001756c60be8'

df_one_ntb = df[df.id == ntb_id]
df_one_ntb.reset_index().to_feather('test_ntb.fth')



In [130]:
df_one_ntb

,index,id,cell_id,cell_type,source,rank,pct_rank,ancestor_id,parent_id
0,0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many he...,0,0.000000,945aea18,None
1,1,00001756c60be8,448eb224,markdown,**Импортируем необходимые для работы функции и...,1,0.017241,945aea18,None
2,2,00001756c60be8,2a9e43d6,code,import numpy as np\nimport pandas as pd\nimpor...,2,0.034483,945aea18,None
3,3,00001756c60be8,7e2f170a,markdown,**Подключаем предупреждения**,3,0.051724,945aea18,None
4,4,00001756c60be8,038b763d,code,import warnings\nwarnings.filterwarnings('igno...,4,0.068966,945aea18,None
5,5,00001756c60be8,77e56113,markdown,"**Устанавливаем значения, чтобы везде был один...",5,0.086207,945aea18,None
6,6,00001756c60be8,2eefe0ef,code,matplotlib.rcParams.update({'font.size': 14}),6,0.103448,945aea18,None
7,7,00001756c60be8,1ae087ab,markdown,**Задаем функцию для подсчета метрик**,7,0.120690,945aea18,None
8,8,00001756c60be8,0beab1cd,code,"def evaluate_preds(train_true_values, train_pr...",8,0.137931,945aea18,None
9,9,00001756c60be8,8ffe0b25,markdown,**Указываем путь к файлам с данными**,9,0.155172,945aea18,None


In [89]:
df[df.id == 'fffc63ff750064'].groupby('cell_type').count()
# '0001bdd4021779'

,index,id,cell_id,source,rank,pct_rank,ancestor_id,parent_id
cell_type,,,,,,,,
code,22,22,22,22,22,22,22,0
markdown,4,4,4,4,4,4,4,0


In [90]:
# df.groupby('id').count()

In [131]:
df_triplet

,p1,p2,label
0,1. # Data exploration,train_set = pd.read_csv('/kaggle/input/house-p...,1
1,1. # Data exploration,test_pred = rf.predict(test_sample)\n\nsubmiss...,0
2,1. # Data exploration,submission.head(),0
3,# # Data preprocessing,missing_train = train_set.isnull().sum()\nmiss...,0
4,# # Data preprocessing,# Take k features with highest correlation\nk ...,1
5,# # Data preprocessing,# Normalization\nscaler = StandardScaler()\n\n...,0
6,# # Data preprocessing,"\nsubmission.to_csv('house_price_rf.csv', inde...",0
7,# Define and fit model,train_set = pd.read_csv('/kaggle/input/house-p...,0
8,# Define and fit model,"X_train.shape, X_test.shape, test_sample.shape",0
9,# Define and fit model,rf = RandomForestRegressor(n_estimators = 1000...,1


In [135]:
df_triplet = pd.DataFrame(generate_triplet(df_one_ntb), columns = ['p1', 'p2', 'label'])
df_triplet[df_triplet['label'] == 1].count()


100%|██████████| 1/1 [00:00<00:00,  2.76it/s]


p1       25
p2       25
label    25
dtype: int64

In [39]:
df_one_ntb.reset_index().to_feather('test_ntb.fth')


In [134]:
import numpy as np

sampler = PairwiseSampler(Path('test_ntb.fth'), dist_processing='binary', 
                  sample_size=1, asymmetric=True, filtration=('first_md', 'second_cd'))
pairs = sampler.sample()
pairs['distance'] = pairs['score']
# pairs['label'] = (abs(pairs['score']) <= 1).astype(int)
pairs[pairs['score'] == 1].count()

100%|██████████| 1/1 [00:00<00:00, 122.95it/s]


p1          25
p2          25
score       25
distance    25
dtype: int64

In [94]:
pairs.distance.min()

-0.2551569506726457

In [50]:
sampler.df

,level_0,index,id,cell_id,cell_type,source,rank,pct_rank,ancestor_id,parent_id
0,6370512,6370512,fffc63ff750064,5015c300,code,import numpy as np\nimport pandas as pd\nimpor...,0,0.000000,0a1b5b65,None
1,6370513,6370513,fffc63ff750064,411b85d9,markdown,1. # Data exploration,1,0.038462,0a1b5b65,None
2,6370514,6370514,fffc63ff750064,8238198c,code,train_set = pd.read_csv('/kaggle/input/house-p...,2,0.076923,0a1b5b65,None
3,6370515,6370515,fffc63ff750064,f4781d1d,code,sns.distplot(train_set['SalePrice']),3,0.115385,0a1b5b65,None
4,6370516,6370516,fffc63ff750064,b5532930,code,missing_train = train_set.isnull().sum()\nmiss...,4,0.153846,0a1b5b65,None
5,6370517,6370517,fffc63ff750064,e1f223e5,code,missing_test = test_set.isnull().sum()\nmissin...,5,0.192308,0a1b5b65,None
6,6370518,6370518,fffc63ff750064,e7e67119,markdown,# # Data preprocessing,6,0.230769,0a1b5b65,None
7,6370519,6370519,fffc63ff750064,4aaf741d,code,# Take k features with highest correlation\nk ...,7,0.269231,0a1b5b65,None
8,6370520,6370520,fffc63ff750064,7229cce6,code,##### fill NaN values\nfor i in train_set:\n ...,8,0.307692,0a1b5b65,None
9,6370521,6370521,fffc63ff750064,a7fa3628,code,"def label_encoder(df_train, df_test):\n le_...",9,0.346154,0a1b5b65,None


In [113]:
sampler.mapping

defaultdict(<function sampler.PairwiseSampler.def_mapping_value()>,
            {'log': <function sampler.PairwiseSampler.log_dist(distance)>,
             'binary': <function sampler.PairwiseSampler.binary_dist(distance)>,
             'multiclass': <bound method PairwiseSampler.multiclass_dist of <sampler.PairwiseSampler object at 0x7fc08c03b850>>,
             'normalised': <bound method PairwiseSampler.normalised_dist of <sampler.PairwiseSampler object at 0x7fc08c03b850>>,
             'harmonic': <bound method PairwiseSampler.harmonic_sampling of <sampler.PairwiseSampler object at 0x7fc08c03b850>>,
             'first_md': <function sampler.PairwiseSampler.filter_markdown(full_sample, nb_df)>})